In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout, Activation

#json file contents
filter_cols = ["Open", "Close", "Volume_(BTC)", "Volume_(Currency)"]

sample = 20
x_window_size = 50
y_window_size = 1
y_column = "Close"

nbepochs = 4
counter_range = 80
batch_size_ = 100

down = 0

df = pd.DataFrame(columns = ['Timestamp', 'Price', 'Return', 'Action'])
timestamp = []
cur_time = []
btcp = []
btcp_pred = []

xbuy = []
xsell = []
bp = []
sp = []
YY = []
Return = []
action = []
tradeprice = []
close = []
plane = []


dataset = pd.read_csv('../input/coinbaseUSD_1-min_data_2014-12-01_to_2018-06-27.csv')
dataset = dataset[['Timestamp', 'Close', 'Open' ,'High', 'Low', 'Volume_(BTC)', 'Volume_(Currency)', 'Weighted_Price']]
dataset = dataset[::sample]
dataset = dataset.reset_index(drop = True)



counter = 0
buy = 0
for counter in range(counter_range):
    
    up = down + 1150
    print("***counter = ", counter, " /", counter_range, " **down = ", down, " **up = ", up)
    data = dataset[down:up]
    data = data.reset_index(drop = True)
    
    time = data['Timestamp'][-100:].values
    realtime = data['Timestamp']

    
    if (filter_cols):
        # Remove any columns from data that we don't need by getting the difference between cols and filter list
        rm_cols = set(data.columns) - set(filter_cols)
        for col in rm_cols:
            del data[col]
    
    # Convert y-predict column name to numerical index
    y_col = list(data.columns).index(y_column)
    
    x_train = data[:-100].reset_index(drop = True)
    x_test = data[-100:].reset_index(drop = True)
    y_test = data['Close'][-100:].reset_index(drop = True)
    
    #present_price = x_train['Close'][len(x_train)-1]
    
    x_col = x_train.columns
    
    sc_X = MinMaxScaler(feature_range=(0, 1))
    x_train = sc_X.fit_transform(x_train)
    x_test = sc_X.transform(x_test)
    
    #present_price_scaled = x_t['Close'][len(x_train)-1]
    
    x_train = pd.DataFrame(data=x_train, columns=x_col)
    x_test = pd.DataFrame(data=x_test, columns=x_col)
    
    
    y_test_scaled = x_test['Close'][-100:]
    
    num_rows = len(x_train)
    x_data = []
    y_data = []
    i = 0
    while ((i + x_window_size + y_window_size) <= num_rows):
        x_window_data = x_train[i:(i + x_window_size)]
        y_window_data = x_train[(i + x_window_size):(i + x_window_size + y_window_size)]
        
        y_average = np.average(y_window_data.values[:, y_col])
        x_data.append(x_window_data.values)
        y_data.append(y_average)
        i += 1
        
    x_np_arr = np.array(x_data)
    y_np_arr = np.array(y_data)
    
    #=================================================================================
    model = Sequential()
    model.add(LSTM(input_dim = 4, output_dim=20, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(20, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(output_dim = 1))
    model.add(Activation("tanh"))
    model.compile(loss = "mse", optimizer = "Nadam")
    history = model.fit(x_np_arr, y_np_arr, validation_split=0.2, epochs = nbepochs, batch_size = batch_size_)
    
    
    dataset_total = pd.concat((x_train, x_test), axis = 0)
    test = dataset_total[-100-x_window_size:]
    test = test.reset_index(drop = True)
    num_rows = len(test)
    x_data = []
    i = 0
#    while ((i + x_window_size) <= num_rows):
#        print(i)
#        test_window = test[i:(i + x_window_size)]
#        x_data.append(test_window.values)
#        i += 1
    
    for i in range(num_rows-100, num_rows):
        test_window = test[i - x_window_size+1 : i+1]
        x_data.append(test_window.values)       
        
    x_test_arr = np.array(x_data)
    
    y_test_scaled = y_test_scaled[1:].reset_index(drop = True)
    y_test = y_test[1:].reset_index(drop = True)
    
    x_test_arr = x_test_arr[:-1]
    time = time[:-1].reshape(-1,1)
    if counter == 0:
        timestamp = time
    else:
        timestamp = np.concatenate((timestamp, time), axis = 0)
    timestamp = timestamp.astype(int)
    
    predicted_price = model.predict(x_test_arr).reshape(-1)
    predicted_price_df = pd.DataFrame(data = predicted_price, columns = ['Close'])
    
    y_test_scaled_df = pd.DataFrame(y_test_scaled)
    y_test_df = pd.DataFrame(y_test)
    
    #------------------------------------------------------------------------------
    ret_pred = (predicted_price_df['Close'] / x_test['Close'][:-1] - 1) * 10000
    
    #ret_real = (x_test['Close'].shift(-1) / x_test['Close'] - 1) * 10000
    
    #plt.plot(ret_pred, color = 'r')
    #plt.plot(ret_real, color = 'g')
    #plt.show()
    
    timestamp = pd.DataFrame(data = timestamp, columns = ['Timestamp'])
    strength = np.percentile(ret_pred, 99.9)
    
    btcp = y_test.values
    
    
    for var in range(len(ret_pred)):
        
        timeline = counter * 99 + var
        
        if ret_pred[var] > strength and buy == 0: # BUY
            Return.append(ret_pred[var])
            action.append('Bought')
            xbuy.append(timeline)
            #plane.append(timeline)
            bp.append(btcp[var])
            tradeprice.append(btcp[var])
            cur_time.append(time[var][0])
            buy = 1
        
        elif ret_pred[var] < -strength and buy == 1: # SELL
            Return.append(ret_pred[var])
            action.append('Sold')
            xsell.append(timeline)
            #plane.append(timeline)
            sp.append(btcp[var])
            tradeprice.append(btcp[var])
            cur_time.append(time[var][0])
            buy = 0

    YY = np.concatenate((YY, btcp), axis = 0)
    down = down+100

plt.figure(figsize = (20, 10))
ax0 = plt.plot(YY, color = 'orange', linewidth = 0.8)
ax1 = plt.scatter(xbuy, bp, color='g') #buy
ax2 = plt.scatter(xsell, sp, color='r') #sell
plt.legend((ax0, ax1, ax2), ('BTC_Price', 'Bought', 'Sold'))


df['Timestamp'] = cur_time
df['Price'] =  tradeprice
df['Return'] =  Return
df['Action'] = action


In [ ]:
#====================================** TRADE SHEET **======================================================================


TS = pd.DataFrame(columns = ['Timestamp', 'Price','Action','Position','TradeQ',
                             'Cash','BTC_Amt','Cu_Credit','Cu_Debit',
                             'Trade','Turnover','PNL','PNLperTrade'])

pos = []
Action = []
PNL = []
Price = []
tradq = []
trade = []
turn = []
Cash = []
btcamt = []
CC = []
CD = []
p_ = 0
pb_ = 0
ps_ = 0
r = 0
cur_pro = 0
mon = 0
btc = 0
credit = 0
debit =0
cumcredit = 0
cumdebit = 0
q1 = 0
q2 = 0
cur_trade = 0
pre_trade = 0
prev = 0
pnl = 0
Q = 0
time_ = []


for var in range(len(df)):

    btcp = df['Price'][var]
    # Buying
    if df['Action'][var] == 'Bought':
        time_.append(df['Timestamp'][var])
        Action.append('Bought')
        Price.append(btcp)
        p_ = 1
        btc = 1
        pos.append(p_)
        if r == 0:
            q2 = 1
            r = 1
        else:
            q2 = p_
        
        Q = q2-q1
        tradq.append(Q)
        q1 = q2
        mon -= btcp*Q
        Cash.append(mon)
        btcamt.append(p_*btcp)
        debit = (btcp*Q)
        cumdebit += debit
        CD.append(cumdebit)
        CC.append(cumcredit)
        trade.append(credit+debit)
        turn.append(cumcredit+cumdebit)
        pnl = cumcredit - cumdebit + p_*btcp
        PNL.append(pnl)

    # Selling
    if df['Action'][var] == 'Sold':
        time_.append(df['Timestamp'][var])
        Action.append('Sold')
        Price.append(btcp)
        p_ = -1
        btc = 0
        pos.append(p_)
        q2 = p_        
        Q = q2 - q1
        tradq.append(Q)
        q1 = q2
        mon += btcp*(-Q)
        Cash.append(mon)
        btcamt.append(p_*btcp)
        credit = (btcp*(-Q))
        cumcredit += credit
        CC.append(cumcredit)
        CD.append(cumdebit)
        trade.append(credit+debit)
        turn.append(cumcredit+cumdebit)
        pnl = cumcredit - cumdebit + p_*btcp
        PNL.append(pnl)

TS['Timestamp'] = time_
TS['Price'] = Price
TS['Action'] = Action
TS['Position'] = pos

TS['Cash'] = Cash
TS['BTC_Amt'] = btcamt
tradq = pd.Series(tradq)
TS['TradeQ'] = tradq
TS['Cu_Credit'] = CC
TS['Cu_Debit'] = CD

TS['Trade'] = trade
TS['PNL'] = PNL
TS['Turnover'] = turn

TS['PNLperTrade'] = TS['PNL'] / TS['Turnover']
TS['trans_cost'] = TS['Turnover'] * (0.3/100)
TS['PnL_after_TC'] = TS['PNL'] - TS['trans_cost']

plt.figure(figsize = (20, 10))
plt.plot(TS['PnL_after_TC'])
plt.plot(TS['PNL'])
plt.legend()
plt.savefig('PNL.png')
plt.show()

In [ ]:
TS

In [ ]:
TS.to_csv('TS_init.csv', index = False)

In [ ]:
dur = TS['Timestamp'].shift(-1) - TS['Timestamp']

In [ ]:
dur.describe()